In [1]:
import tensorflow as tf
import time
import json
tf.__version__

'2.0.0'

In [2]:
with open('./train.json','r') as f:
    train_txt = json.load(f)

In [3]:
train_txt

{'0 1 2 3 ': '53024 3 1 4 5 53025',
 '6 7 8 8 8 9 10 11 3 ': '53024 3 12 5 12 5 12 5 13 14 15 16 53025',
 '17 18 19 20 21 22 23 24 17 25 26 27 3 ': '53024 3 28 29 30 31 32 33 34 35 12 53025',
 '36 37 38 39 40 41 42 3 ': '53024 3 43 44 45 46 21 47 48 49 50 3 53025',
 '51 13 52 3 ': '53024 3 53 53 21 54 55 11453 53025',
 '52163 57 3 ': '53024 3 58 59 60 33 61 62 23 30 53025',
 '41 63 64 65 3 ': '53024 3 66 34 67 68 32 53025',
 '1 69 70 71 72 17 13 17 38 34 72 71 3 ': '53024 3 71 73 74 75 76 21 77 78 79 17 80 16 21 81 82 17 83 84 85 23 53025',
 '86 87 3 ': '53024 3 88 19 1237 53025',
 '3443 90 3 ': '53024 3 91 17 92 90 71 21 71 93 94 95 32 53025',
 '30 34 96 3 ': '53024 3 97 65 98 8 53025',
 '99 100 3 ': '53024 3 101 99 102 103 5 21 104 105 104 106 107 108 71 109 32 53025',
 '110 111 112 113 114 3 ': '53024 3 110 17 113 115 53025',
 '116 117 30 118 3 ': '53024 3 116 64 116 21 119 64 119 21 120 121 64 39 122 23 53025',
 '123 3 ': '53024 3 34 124 107 39 125 126 127 53025',
 '128 121 129 130

In [4]:
with open('./word2id.json','r') as f:
    word2id = json.load(f)
word2id['<pad>']

53026

In [5]:
with open('./id2word.json','r') as f:
    id2word = json.load(f)
id2word['53026']

'<pad>'

In [6]:
id2word = {int(k):v for k,v in id2word.items()}
id2word[53026]

'<pad>'

In [7]:
ask = []
answer =[]
for k,v in train_txt.items():
    k = k.split()
    k = [int(i) for i in k]
    ask.append(k)
    v = v.split()
    v = [int(i) for i in v]
    answer.append(v)
ask[0]

[0, 1, 2, 3]

In [8]:
len(answer)

114969

In [9]:
def toSentence(sentence):
    s = [id2word[i] for i in sentence]
    return s
toSentence(ask[0])

['南京', '在', '哪里', ' ']

In [10]:
toSentence(answer[0])

['<start>', ' ', '在', '这里', '了', '<end>']

In [11]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(
    ask, value=word2id['<pad>'],
    padding='post', maxlen=20
)

In [12]:
x_train[0]

array([    0,     1,     2,     3, 53026, 53026, 53026, 53026, 53026,
       53026, 53026, 53026, 53026, 53026, 53026, 53026, 53026, 53026,
       53026, 53026])

In [13]:
toSentence(x_train[0])

['南京',
 '在',
 '哪里',
 ' ',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [14]:
x_train.shape

(114969, 20)

In [15]:
import numpy as np
y_train = tf.keras.preprocessing.sequence.pad_sequences(
    answer,value=word2id['<pad>'],
    padding='post', maxlen=20
)

In [16]:
y_train.shape

(114969, 20)

In [17]:
toSentence(y_train[0])

['<start>',
 ' ',
 '在',
 '这里',
 '了',
 '<end>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [18]:
x_train.shape

(114969, 20)

In [19]:
y_train.shape

(114969, 20)

In [20]:
BUFFER_SIZE = len(x_train)
BATCH_SIZE = 64
steps_per_epoch = len(x_train)//BATCH_SIZE
embedding_dim = 100
units = 512
vocab_size = len(word2id)+1

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [21]:
for i in dataset.take(2):
    print(i)

(<tf.Tensor: id=20, shape=(64, 20), dtype=int32, numpy=
array([[   34,   286,   104, ..., 53026, 53026, 53026],
       [  165,  2819,     5, ..., 53026, 53026, 53026],
       [12116, 10404,    21, ..., 53026, 53026, 53026],
       ...,
       [ 2980,    64,   423, ..., 53026, 53026, 53026],
       [  967,  4510, 28848, ..., 53026, 53026, 53026],
       [  613,    71,    65, ..., 53026, 53026, 53026]])>, <tf.Tensor: id=21, shape=(64, 20), dtype=int32, numpy=
array([[53024,     3,    59, ..., 53026, 53026, 53026],
       [53024,     3,    71, ..., 53026, 53026, 53026],
       [53024,     3,  1087, ..., 53026, 53026, 53026],
       ...,
       [   43,   884,    21, ...,   360,    23, 53025],
       [53024,     3,    80, ..., 53026, 53026, 53026],
       [53024,     3,    17, ..., 53026, 53026, 53026]])>)
(<tf.Tensor: id=22, shape=(64, 20), dtype=int32, numpy=
array([[   71,    34,   761, ..., 53026, 53026, 53026],
       [   17,   883,  2751, ..., 53026, 53026, 53026],
       [   17,   57

In [22]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 20]), TensorShape([64, 20]))

In [23]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.encoder_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden) #全部隐藏状态，最后一个隐藏状态
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.encoder_units))

In [24]:
encoder = Encoder(vocab_size, embedding_dim, units, BATCH_SIZE)

# 样本输入
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 20, 512)
Encoder Hidden state shape: (batch size, units) (64, 512)


In [25]:
class Attention(tf.keras.layers.Layer):
    def __init__(self,units):
        super(Attention,self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
    
    def call(self,query,values):
        hidden_with_the_time_axis = tf.expand_dims(query,1)
        score = self.V(tf.nn.tanh(self.W1(values) + 
                                 self.W2(hidden_with_the_time_axis)))
        
        attention_weights = tf.nn.softmax(score,axis=1)
        
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector,axis=1)
        
        return context_vector , attention_weights

In [26]:
w1 = tf.keras.layers.Dense(10)
w2 = tf.keras.layers.Dense(10)
v = tf.keras.layers.Dense(1)
hidden_with_time_axis = tf.expand_dims(sample_hidden,1)
hidden_with_time_axis

<tf.Tensor: id=154, shape=(64, 1, 512), dtype=float32, numpy=
array([[[ 0.01017231,  0.01381851,  0.00506639, ..., -0.01406328,
         -0.01437947,  0.00515292]],

       [[ 0.01022554,  0.01392362,  0.00506379, ..., -0.01392079,
         -0.01446057,  0.00526271]],

       [[ 0.01023075,  0.01393857,  0.00507667, ..., -0.01392326,
         -0.01446573,  0.00526784]],

       ...,

       [[ 0.01012564,  0.01392819,  0.00509037, ..., -0.01402845,
         -0.01440333,  0.00521117]],

       [[ 0.01023976,  0.01394498,  0.00506557, ..., -0.01389837,
         -0.01448352,  0.00527214]],

       [[ 0.01022419,  0.01391986,  0.00507922, ..., -0.01392085,
         -0.0144555 ,  0.00526389]]], dtype=float32)>

In [27]:
score = v(tf.nn.tanh(w1(sample_output) + w2(hidden_with_time_axis)))
print(score.shape)
attention_weights = tf.nn.softmax(score, axis=1)
print(attention_weights.shape)

(64, 20, 1)
(64, 20, 1)


In [28]:
tf.reduce_sum(attention_weights * sample_output,axis = 1)

<tf.Tensor: id=298, shape=(64, 512), dtype=float32, numpy=
array([[ 0.00555823,  0.00899295, -0.00040581, ..., -0.00378942,
        -0.00864212,  0.00283751],
       [ 0.00458268,  0.01171986, -0.00025007, ..., -0.00799214,
        -0.01170614,  0.00614353],
       [ 0.00682287,  0.01178445,  0.00297119, ..., -0.00934313,
        -0.01026499,  0.00560975],
       ...,
       [ 0.0042644 ,  0.01062403,  0.00139395, ..., -0.00684166,
        -0.0087492 ,  0.00406666],
       [ 0.00796507,  0.01066431,  0.00377895, ..., -0.00952938,
        -0.01346969,  0.00494389],
       [ 0.00722906,  0.01194496,  0.00306622, ..., -0.00854686,
        -0.01158864,  0.00434941]], dtype=float32)>

In [29]:
attention_layer = Attention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 512)
Attention weights shape: (batch_size, sequence_length, 1) (64, 20, 1)


In [30]:
class Decoder(tf.keras.Model):
    def __init__(self,vocab_size,embedding_dim,decoder_units,batch_size):
        super(Decoder,self).__init__()
        self.batch_size = batch_size
        self.decoder_units = decoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
        self.gru = tf.keras.layers.GRU(self.decoder_units,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = Attention(self.decoder_units)
    
    def call(self,x,hidden,encoder_output):
        context_vector, attention_weights = self.attention(hidden,encoder_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector,1),x],axis=-1)
        output, state = self.gru(x)
        
        output = tf.reshape(output,(-1,output.shape[2]))
        
        x = self.fc(output)
        
        return x,state,attention_weights

In [31]:
decoder = Decoder(vocab_size,embedding_dim,units,BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)),
                                      sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 53028)


In [32]:
optimizer = tf.keras.optimizers.RMSprop()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')

def loss_function(real,pred):
    mask = tf.math.logical_not(tf.math.equal(real,0))
    loss_ = loss_object(real,pred)
    
    mask = tf.cast(mask,dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

In [33]:
@tf.function
def train_step(inputs,targets,encoder_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        encoder_output , encoder_hidden = encoder(inputs,encoder_hidden)
        
        decoder_hidden = encoder_hidden
        
        decoder_input = tf.expand_dims([word2id["<start>"]]*BATCH_SIZE,1)
        
        for t in range(1,targets.shape[1]):
            predictions ,decoder_hidden , _ = decoder(decoder_input,decoder_hidden,encoder_hidden)
            loss += loss_function(targets[:,t],predictions)
            
            decoder_input = tf.expand_dims(targets[:,t],1)
        
        batch_loss = (loss/int(targets.shape[1]))
        
        variables = encoder.trainable_variables + decoder.trainable_variables
        
        gradients = tape.gradient(loss,variables)
        
        optimizer.apply_gradients(zip(gradients,variables))
        
        return batch_loss

In [34]:
epochs = 10

for epoch in range(epochs):
    start = time.time()
    
    encoder_hidden = encoder.initialize_hidden_state()
    
    total_loss = 0
    
    for (batch, (inputs,targets)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inputs,targets,encoder_hidden)
        total_loss += batch_loss
        
        if batch % 100 == 0:
            print("Epochs {}  Batch{} Loss{:.4f}".format(epoch + 1,batch,batch_loss.numpy()))
        
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epochs 1  Batch0 Loss10.3346
Epochs 1  Batch100 Loss2.2385
Epochs 1  Batch200 Loss2.1315
Epochs 1  Batch300 Loss1.9298
Epochs 1  Batch400 Loss1.9422
Epochs 1  Batch500 Loss2.1217
Epochs 1  Batch600 Loss1.8889
Epochs 1  Batch700 Loss1.9973
Epochs 1  Batch800 Loss1.9274
Epochs 1  Batch900 Loss1.6027
Epochs 1  Batch1000 Loss2.0929
Epochs 1  Batch1100 Loss1.8619
Epochs 1  Batch1200 Loss1.5912
Epochs 1  Batch1300 Loss1.5394
Epochs 1  Batch1400 Loss1.4286
Epochs 1  Batch1500 Loss1.9838
Epochs 1  Batch1600 Loss1.6386
Epochs 1  Batch1700 Loss1.6048
Epoch 1 Loss 1.9684
Time taken for 1 epoch 691.7735021114349 sec

Epochs 2  Batch0 Loss1.6929
Epochs 2  Batch100 Loss1.6355
Epochs 2  Batch200 Loss1.5854
Epochs 2  Batch300 Loss1.7105
Epochs 2  Batch400 Loss1.4222
Epochs 2  Batch500 Loss1.4036
Epochs 2  Batch600 Loss1.6879
Epochs 2  Batch700 Loss1.4424
Epochs 2  Batch800 Loss1.6976
Epochs 2  Batch900 Loss1.9347
Epochs 2  Batch1000 Loss1.6353
Epochs 2  Batch1100 Loss1.7840
Epochs 2  Batch1200 Loss1.7

In [150]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)
checkpoint.save(file_prefix = checkpoint_prefix)

'./training_checkpoints\\ckpt-1'

In [145]:
encoder.save_weights('./encoder_weights.h5')

In [146]:
decoder.save_weights('./decoder_weights.h5')

In [123]:
import jieba
def predict(sentence):
    for ch in "#@$%^&*():;：；{}[]'_<>-+/~0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ":
        sentence = sentence.replace(ch,"")
    sentence = jieba.cut(sentence)
    sentence = [word2id.get(word,word2id["<pad>"]) for word in sentence]
    sentence = tf.keras.preprocessing.sequence.pad_sequences([sentence],value=word2id["<pad>"],
                                                             maxlen=20,padding='post')
    sentence = tf.convert_to_tensor(sentence)
    
    result = ''
    hidden = [tf.zeros((1,units))] #现在只输入了一个所以batch_size是1
    encoder_output, encoder_hidden = encoder(sentence,hidden)
    
    decoder_hidden = encoder_hidden
    decoder_input = tf.expand_dims([word2id["<start>"]],0)
    
    for t in range(20):
        predictions , decoder_hidden , attention_weights = decoder(decoder_input,
                                                                   decoder_hidden,
                                                                   encoder_output)
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        if id2word[predicted_id] == "<end>":
            return result
        result += id2word[predicted_id]
        
        decoder_input = tf.expand_dims([predicted_id],0)
    
    return result

In [124]:
predict("百度")

' 你说的什么哦？换种说法行不'

In [149]:
encoder2 = Encoder(vocab_size, embedding_dim, units, BATCH_SIZE)
decoder2 = Decoder(vocab_size,embedding_dim,units,BATCH_SIZE)
encoder2.load_weights('./encoder_weights.tf.data-00000-of-00002',by_name=True)
decoder2.load_weights('./decoder_weights.tf.index',by_name=True)

OSError: Unable to open file (file signature not found)

In [142]:
def predict(sentence):
    for ch in "#@$%^&*():;：；{}[]'_<>-+/~0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ":
        sentence = sentence.replace(ch,"")
    sentence = jieba.cut(sentence)
    sentence = [word2id.get(word,word2id["<pad>"]) for word in sentence]
    sentence = tf.keras.preprocessing.sequence.pad_sequences([sentence],value=word2id["<pad>"],
                                                             maxlen=20,padding='post')
    sentence = tf.convert_to_tensor(sentence)
    
    result = ''
    hidden = [tf.zeros((1,units))] #现在只输入了一个所以batch_size是1
    encoder_output, encoder_hidden = encoder2(sentence,hidden)
    
    decoder_hidden = encoder_hidden
    decoder_input = tf.expand_dims([word2id["<start>"]],0)
    
    for t in range(20):
        predictions , decoder_hidden , attention_weights = decoder2(decoder_input,
                                                                   decoder_hidden,
                                                                   encoder_output)
        print(predictions.shape)
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        if id2word[predicted_id] == "<end>":
            return result
        result += id2word[predicted_id]
        
        decoder_input = tf.expand_dims([predicted_id],0)
    
    return result

In [143]:
predict("")

(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)
(1, 53028)


'焦嘉城秦王面書贱点曹阳东琪吉人天相校招驕椅海枯石烂沒奶方力申组织部磨发言人开度百么爸会號碼秦王'